#### Load LLM and device

In [1]:
from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(model="gemini-1.5-flash")

In [2]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using cuda device


#### LongForm UQ

In [3]:
from uqlm import LongFormUQ

In [4]:
prompts = ["write a paragraph about Paul McCartney", "write a paragraph about John Lennon"]
sent_scorers = ["response_sent_entail", "response_sent_noncontradict", "response_sent_contrast_entail"]
claim_scorers = ["response_claim_entail", "response_claim_noncontradict", "response_claim_contrast_entail"]
matched_claim_scorers = ["matched_claim_entail", "matched_claim_noncontradict", "matched_claim_contrast_entail"]

In [5]:
# Sentence Level UQ
sent_lfuq = LongFormUQ(llm=llm, scorers=sent_scorers, device=device)
result = await sent_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

,response,sampled_responses,prompt,sentence_set,response_sent_entail,response_sent_noncontradict,response_sent_contrast_entail
0,"Sir Paul McCartney, a pivotal figure in the hi...","[Sir James Paul McCartney, a towering figure i...",write a paragraph about Paul McCartney,"[Sir Paul McCartney, a pivotal figure in the h...",0.664817,0.998687,0.995217
1,"John Lennon, a founding member of the Beatles,...","[John Lennon, a pivotal figure in the 20th cen...",write a paragraph about John Lennon,"[John Lennon, a founding member of the Beatles...",0.578596,0.998184,0.991760


In [6]:
# Claim Level UQ
claim_lfuq = LongFormUQ(llm=llm, scorers=claim_scorers, device=device)
result = await claim_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

,response,sampled_responses,prompt,claim_set,response_claim_entail,response_claim_noncontradict,response_claim_contrast_entail
0,"Sir James Paul McCartney, a pivotal figure in ...","[Sir James Paul McCartney, a pivotal figure in...",write a paragraph about Paul McCartney,[Sir James Paul McCartney is a pivotal figure ...,0.520694,0.994923,0.909801
1,"John Lennon, a pivotal figure in the 20th cent...","[John Lennon, a founding member of the Beatles...",write a paragraph about John Lennon,"[John Lennon was a pivotal figure.\n, John Len...",0.746598,0.997056,0.955162


In [7]:
# matched claim UQ
matched_claim_lfuq = LongFormUQ(llm=llm, scorers=matched_claim_scorers, device=device)
result = await matched_claim_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

,response,sampled_responses,prompt,claim_set,matched_claim_entail,matched_claim_noncontradict,matched_claim_contrast_entail
0,"Sir James Paul McCartney, a pivotal figure in ...","[Sir Paul McCartney, a towering figure in musi...",write a paragraph about Paul McCartney,[Sir James Paul McCartney is a pivotal figure....,0.524010,0.998736,0.958228
1,"John Lennon, a founding member of the Beatles,...","[John Lennon, a pivotal figure in the 20th cen...",write a paragraph about John Lennon,[John Lennon was a founding member of the Beat...,0.540962,0.999304,0.984766


#### Decomposition

In [8]:
from uqlm.longform import ResponseDecomposer

In [9]:
rd = ResponseDecomposer(llm)

In [14]:
responses = [
    "Hello there! Today I visited Dr. Chauhan at his home. He was hanging his framed B.S., M.S., and Ph.D. degrees. It was fun!",
    "One time I went on a class trip to D.C. to visit the white house. While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others."
]

sampled_responses = [
    [
        "Hi! I went to see Dr. Chauhan at his house today. He was busy putting up his framed B.S., M.S., and Ph.D. diplomas. It was quite enjoyable!",
        "Greetings! Today, I stopped by Dr. Chauhan's home. He was in the process of displaying his framed degrees: B.S., M.S., and Ph.D. It was a delightful experience!"
    ],
    [
        "Once, I took a school trip to Washington, D.C. to see the White House. While I was there, I was amazed by the stunning artwork and the notable individuals with titles like Professor, Esquire, and more.",
        "During a class excursion to Washington, D.C., I had the opportunity to visit the White House. I was struck by the beautiful art and the distinguished people holding titles such as Prof., Esq., and others."
    ]
    
]

In [11]:
# Decompose into sentences
rd.decompose_sentences(responses=responses)

[['Hello there!',
  'Today I visited Dr. Chauhan at his home.',
  'He was hanging his framed B.S., M.S., and Ph.D. degrees.',
  'It was fun!'],
 ['One time I went on a class trip to D.C. to visit the white house.',
  'While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others.']]

In [12]:
# Decompose into claims
await rd.decompose_claims(responses=responses)

[['I visited Dr. Chauhan.\n',
  'I visited Dr. Chauhan at his home.\n',
  'Dr. Chauhan was hanging his degrees.\n',
  'Dr. Chauhan was hanging his B.S. degree.\n',
  'Dr. Chauhan was hanging his M.S. degree.\n',
  'Dr. Chauhan was hanging his Ph.D. degree.\n',
  "Dr. Chauhan's degrees were framed.\n",
  'It was fun.\n'],
 ['I went on a class trip.\n',
  'The class trip was to D.C.\n',
  'I visited the white house.\n',
  'I felt in awe.\n',
  'The art was beautiful.\n',
  'The people were important.\n',
  'The people had titles.\n',
  'The titles included Prof..\n',
  'The titles included Esq..\n',
  'The titles included others.\n\n']]

In [15]:
# Decompose sampled responses into claims
await rd.decompose_candidate_claims(sampled_responses=sampled_responses)

[[['I went to see Dr. Chauhan.\n',
   'I went to see Dr. Chauhan at his house.\n',
   'I went to see Dr. Chauhan today.\n',
   'Dr. Chauhan was busy.\n',
   'Dr. Chauhan was putting up diplomas.\n',
   'Dr. Chauhan was putting up framed diplomas.\n',
   "Dr. Chauhan's diplomas were framed.\n",
   "Dr. Chauhan's diplomas were B.S. diplomas.\n",
   "Dr. Chauhan's diplomas were M.S. diplomas.\n",
   "Dr. Chauhan's diplomas were Ph.D. diplomas.\n",
   'It was enjoyable.\n'],
  ["I stopped by Dr. Chauhan's home.\n",
   'Dr. Chauhan was displaying his framed degrees.\n',
   'Dr. Chauhan has a B.S. degree.\n',
   'Dr. Chauhan has an M.S. degree.\n',
   'Dr. Chauhan has a Ph.D. degree.\n',
   'The experience was delightful.\n']],
 [['I took a school trip.\n',
   'The school trip was to Washington, D.C.\n',
   'I went to see the White House.\n',
   'I was amazed by the artwork.\n',
   'The artwork was stunning.\n',
   'I was amazed by the individuals.\n',
   'The individuals were notable.\n',
 